## Part IV:  Features and Labeling Functions

## Generating Features

In [1]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ labels snorkel.db');

In [2]:
# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [3]:
# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`

In [4]:
# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

### Creating feature matrix

In [5]:
# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

In [6]:
# %time F_train = feature_manager.create(session, train, 'Train Features')

In [7]:
# %time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

### Reloading feature matrix

In [8]:
# %time F_train = feature_manager.load(session, train, 'Train Features')

In [9]:
# F_train

In [10]:
# F_dev

In [11]:
# F_train.get_candidate(0)

In [12]:
# F_train.get_key(0)

In [13]:
# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(1))[:10]: print f

### Saving feature matrices

In [14]:
# # If necessary:
# import os
# os.system('cp snorkel.db snorkel.db\ featurized');

## Applying LFs

In [15]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ featurized snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [16]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

### Defining Labeling Functions

In [17]:
from snorkel.lf_helpers import *
from itertools import chain

LFs = []

# POSITIVE

def LF_to_right(c):
    return 1 if 'to' in get_right_ngrams(c.temp, window=1) else 0
LFs.append(LF_to_right)

def LF_150_right(c):
    return 1 if '150' in get_right_ngrams(c.temp, window=4) else 0
LFs.append(LF_150_right)

def LF_storage_row(c):
    return 1 if 'storage' in get_row_ngrams(c.temp) else 0
LFs.append(LF_storage_row)

def LF_operating_row(c):
    return 1 if 'operating' in get_row_ngrams(c.temp) else 0
LFs.append(LF_operating_row)

# def LF_C_phrase(c):
#     ngrams =  get_phrase_ngrams(c.temp, n_min=2, n_max=2, lower=False)
#     return 1 if ('% C' in ngrams or '%C' in ngrams) else 0
# LFs.append(LF_C_phrase)

def LF_temperature_row(c):
    return 1 if 'temperature' in get_row_ngrams(c.temp) else 0
LFs.append(LF_temperature_row)

def LF_tstg_row(c):
    row_ngrams = get_row_ngrams(c.temp)
    return 1 if ('tstg' in row_ngrams or 
                 'ts' in row_ngrams or 
                 'stg' in row_ngrams) else 0
LFs.append(LF_tstg_row)

# NEGATIVE

def LF_not_temp_relevant(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if not ('storage' in ngrams or
                      'temperature' in ngrams or
                      'tstg' in ngrams or
                      'ts' in ngrams or
                      'stg' in ngrams) else 0
LFs.append(LF_not_temp_relevant)

def LF_temp_outside_table(c):
    return -1 if c.temp.parent.row is None else 0
LFs.append(LF_temp_outside_table)

def LF_too_many_numbers_row(c):
    num_numbers = list(get_row_ngrams(c.temp, attrib="ner_tags")).count('number')
    return -1 if num_numbers >= 3 else 0
LFs.append(LF_too_many_numbers_row)

def LF_collector_aligned(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if (
        'collector'         in ngrams or
        'collector-current' in ngrams or
        'collector-base'    in ngrams or
        'collector-emitter' in ngrams) else 0
LFs.append(LF_collector_aligned)

def LF_current_aligned(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if (
        'current' in ngrams or
        'dc'      in ngrams or
        'ic'      in ngrams) else 0    
LFs.append(LF_current_aligned)

def LF_voltage_row_temp(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if (
        'voltage' in ngrams or
        'cbo'     in ngrams or
        'ceo'     in ngrams or
        'ebo'     in ngrams or
        'v'       in ngrams) else 0
LFs.append(LF_voltage_row_temp)

def LF_voltage_row_part(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if (
        'voltage' in ngrams or
        'cbo'     in ngrams or
        'ceo'     in ngrams or
        'ebo'     in ngrams or
        'v'       in ngrams) else 0
LFs.append(LF_voltage_row_part)

def LF_typ_row(c):
    ngrams = get_row_ngrams(c.temp)
    return -1 if ('typ' in ngrams or
                  'typ.' in ngrams) else 0
LFs.append(LF_typ_row)

def LF_test_condition_aligned(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if ('test'      in ngrams or
                  'condition' in ngrams) else 0
LFs.append(LF_test_condition_aligned)

def LF_complement_left_row(c):
    return -1 if 'complement' in get_row_ngrams(c.part) else 0
LFs.append(LF_complement_left_row)

In [ ]:
# LF2s = []
# from snorkel.models import AnnotationKey
# from snorkel.utils import get_ORM_instance

# %time F_train = feature_manager.load(session, train, 'Train Features')

# def feature_lookup(candidate, entity, ngram, helper, attrib='words'):
#     if helper == 'get_row_ngrams':
#         feature = entity + "_ROW_%s_[%s]" % (attrib.upper(), ngram)
#         key = get_ORM_instance(AnnotationKey, session, feature)
#     return F_train[get_row_index(candidate), get_col_index(key)]
    
# def LF2_storage_row(c):
#     return 1 if feauture_lookup(c, 'e1', 'storage', 'get_row_ngrams') else 0
# LF2s.append(LF_storage_row)

# def LF2_operating_row(c):
#     return 1 if 'operating' in get_row_ngrams(c.temp) else 0
# LF2s.append(LF_operating_row)


### Testing LFs and LF_helpers

In [18]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
c = train[12]
print c.part
print c.temp
print c.temp.parent
print c.temp.parent.table
print same_document(c)
print same_table(c)
print contains_regex(c, rgx=r'849')


ImplicitSpan("BC817W", parent=99496, words=[4,4], position=[7])
ImplicitSpan("-65", parent=99136, words=[0,1], position=[0])
Phrase(Doc: PHGLS20126-1, Table: 13, Row: 13, Col: 6, Position: 0, Text: - 65)
Table(Doc: PHGLS20126-1, Position: 13)
True
False
False


In [19]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

Generating annotations for 85377 candidates...
[========================================] 100%

Loading sparse Label matrix...
CPU times: user 27min 18s, sys: 4min 29s, total: 31min 48s
Wall time: 32min 37s


<85377x16 sparse matrix of type '<type 'numpy.float64'>'
	with 256307 stored elements in Compressed Sparse Row format>

In [20]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [21]:
# TEMP
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ features snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()

# train_gold = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

# from snorkel.annotations import LabelManager
# label_manager = LabelManager()
# %time L_train = label_manager.load(session, train, 'LF Labels')
# L_train
# TEMP

In [22]:
# L_train.lf_stats()
%time L_train.lf_stats(train_gold)

[========================================] 100%

CPU times: user 1.04 s, sys: 94.3 ms, total: 1.14 s
Wall time: 1.14 s


,accuracy,conflicts,coverage,j,overlaps
LF_to_right,1.000000,0.023133,0.149806,0,0.149806
LF_150_right,1.000000,0.014945,0.094674,1,0.094674
LF_storage_row,1.000000,0.027560,0.256697,2,0.256697
LF_operating_row,1.000000,0.027209,0.108015,3,0.108015
LF_temperature_row,1.000000,0.076086,0.302833,4,0.302833
LF_tstg_row,1.000000,0.008714,0.035326,5,0.035326
LF_not_temp_relevant,0.788500,0.048912,0.743303,6,0.632407
LF_temp_outside_table,0.671078,0.032093,0.384155,7,0.384155
LF_too_many_numbers_row,0.891062,0.003713,0.162889,8,0.162889
LF_collector_aligned,0.806961,0.000000,0.145379,9,0.145379


In [23]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

Next, in Part 5, we will test our model on the development `CandidateSet`.

# TEMPORARY

In [24]:
# print c.temp
# print c.temp.parent

In [25]:
# print list(get_row_ngrams(c.temp, attrib="ner_tags"))
# print list(get_row_ngrams(c.temp, attrib="words"))

In [26]:
# from pprint import pprint
# print(list(get_row_ngrams(c.temp, attrib="ner_tags")).count('o'))

In [27]:
# print LF_v_row(c)